In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [54]:
# 建立資料夾
os.makedirs("data", exist_ok=True)

In [55]:
# 基本設定
base_url = "https://www.ptt.cc"
board = "CheerGirlsTW"
start_url = f"{base_url}/bbs/{board}/index.html"
page_count = 3  # 要爬的頁數

In [56]:
# 繞過年齡驗證
session = requests.Session()
session.cookies.set("over18", "1")

Cookie(version=0, name='over18', value='1', port=None, port_specified=False, domain='', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)

In [57]:
def get_articles_from_page(url):
    res = session.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    entries = soup.select("div.r-ent")
    page_articles = []

    for e in entries:
        try:
            push = e.select_one("div.nrec").text.strip()
            title_tag = e.select_one("div.title a")
            title = title_tag.text.strip()
            link = base_url + title_tag["href"]
            author = e.select_one("div.author").text.strip()
            page_articles.append([title, author, push, link])
        except:
            continue  # 跳過已被刪除的文章

    return page_articles, soup

In [58]:
# 開始爬取
all_articles = []
current_url = start_url

for i in range(page_count):
    print(f"🔍 正在爬第 {i+1} 頁：{current_url}")
    articles, soup = get_articles_from_page(current_url)
    all_articles.extend(articles)

    # 找到上一頁連結
    prev_link = soup.select("div.btn-group-paging a")[-1]["href"]
    current_url = base_url + prev_link

🔍 正在爬第 1 頁：https://www.ptt.cc/bbs/CheerGirlsTW/index.html
🔍 正在爬第 2 頁：https://www.ptt.cc/bbs/CheerGirlsTW/index.html
🔍 正在爬第 3 頁：https://www.ptt.cc/bbs/CheerGirlsTW/index.html


In [59]:
# 儲存 CSV
df = pd.DataFrame(all_articles, columns=["標題", "作者", "推文", "連結"])
df.to_csv("data/cheergirls_articles_3pages.csv", index=False, encoding="utf-8-sig")

In [60]:
print("✅ 完成！共爬到", len(df), "篇文章，已儲存到 data/cheergirls_articles_3pages.csv")

✅ 完成！共爬到 27 篇文章，已儲存到 data/cheergirls_articles_3pages.csv
